In [1]:
def setupSpark():
  # Spark needs to run with Java 8 ... 
  !pip install -q findspark
  !apt-get install openjdk-8-jdk-headless > /dev/null
  !echo 2 | update-alternatives --config java > /dev/null
  !java -version
  import os, findspark
  os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
  # !echo JAVA_HOME=$JAVA_HOME
  !pip install -q pyspark
  findspark.init(spark_home='/usr/local/lib/python3.7/dist-packages/pyspark')
  !pyspark --version


setupSpark()

openjdk version "1.8.0_292"
OpenJDK Runtime Environment (build 1.8.0_292-8u292-b10-0ubuntu1~18.04-b10)
OpenJDK 64-Bit Server VM (build 25.292-b10, mixed mode)
     |████████████████████████████████| 212.4 MB 64 kB/s 
     |████████████████████████████████| 198 kB 54.2 MB/s 
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.1.2
      /_/
                        
Using Scala version 2.12.10, OpenJDK 64-Bit Server VM, 1.8.0_292
Branch HEAD
Compiled by user centos on 2021-05-24T04:27:48Z
Revision de351e30a90dd988b133b3d00fa6218bfcaba8b8
Url https://github.com/apache/spark
Type --help for more information.


In [2]:
from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession
conf=SparkConf()
SpContext=SparkContext(conf=conf)
SpSession=SparkSession(SpContext)

In [3]:
bankData = SpContext.textFile("bank.csv")
bankData.cache()
bankData.count()


11163

In [4]:
firstLine=bankData.first()
dataLines = bankData.filter(lambda x: x != firstLine)
dataLines.count()


11162

In [6]:
import math
from pyspark.ml.linalg import Vectors
from pyspark.sql import Row
def transformToNumeric( inputStr) :
    
    attList=inputStr.replace("\"","").split(",")
    
    age=float(attList[0])
    #convert outcome to float    
    outcome = 0.0 if attList[16] == "no" else 1.0
    
    #create indicator variables for single/married    
    single= 1.0 if attList[2] == "single" else 0.0
    married = 1.0 if attList[2] == "married" else 0.0
    divorced = 1.0 if attList[2] == "divorced" else 0.0
    
    #create indicator variables for education
    primary = 1.0 if attList[3] == "primary" else 0.0
    secondary = 1.0 if attList[3] == "secondary" else 0.0
    tertiary = 1.0 if attList[3] == "tertiary" else 0.0
    
    #convert default to float
    default= 0.0 if attList[4] == "no" else 1.0
    #convert balance amount to float
    balance=float(attList[5])
    #convert loan to float
    loan= 0.0 if attList[7] == "no" else 1.0
    
    #Create a row with cleaned up and converted data
    values= Row(     OUTCOME=outcome ,\
                    AGE=age, \
                    SINGLE=single, \
                    MARRIED=married, \
                    DIVORCED=divorced, \
                    PRIMARY=primary, \
                    SECONDARY=secondary, \
                    TERTIARY=tertiary, \
                    DEFAULT=default, \
                    BALANCE=balance, \
                    LOAN=loan                    
                    ) 
    return values
    
#Change to a Vector
bankRows = dataLines.map(transformToNumeric)
bankRows.collect()[:15]

bankData = SpSession.createDataFrame(bankRows)


In [7]:
bankData.describe().show()

#Find correlation between predictors and target
for i in bankData.columns:
    if not( isinstance(bankData.select(i).take(1)[0][0], str)) :
        print( "Correlation to OUTCOME for ", i, \
            bankData.stat.corr('OUTCOME',i))


+-------+------------------+------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+
|summary|           OUTCOME|               AGE|             SINGLE|           MARRIED|           DIVORCED|            PRIMARY|          SECONDARY|           TERTIARY|            DEFAULT|           BALANCE|              LOAN|
+-------+------------------+------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+
|  count|             11162|             11162|              11162|             11162|              11162|              11162|              11162|              11162|              11162|             11162|             11162|
|   mean|0.4738398136534671|41.231947679627304| 0.3151764916681598|0.5689840530370901|0.115839455294

In [8]:
def transformToLabeledPoint(row) :
    lp = ( row["OUTCOME"], \
            Vectors.dense([
                row["AGE"], \
                row["BALANCE"], \
                row["DEFAULT"], \
                row["DIVORCED"], \
                row["LOAN"], \
                row["MARRIED"], \
                row["PRIMARY"], \
                row["SECONDARY"], \
                row["SINGLE"], \
                row["TERTIARY"]
        ]))
    return lp
    
bankLp = bankData.rdd.map(transformToLabeledPoint)
bankLp.collect()
bankDF = SpSession.createDataFrame(bankLp,["label", "features"])
bankDF.select("label","features").show(10)


+-----+--------------------+
|label|            features|
+-----+--------------------+
|  1.0|[59.0,2343.0,0.0,...|
|  1.0|[56.0,45.0,0.0,0....|
|  1.0|[41.0,1270.0,0.0,...|
|  1.0|[55.0,2476.0,0.0,...|
|  1.0|[54.0,184.0,0.0,0...|
|  1.0|[42.0,0.0,0.0,0.0...|
|  1.0|[56.0,830.0,0.0,0...|
|  1.0|[60.0,545.0,0.0,1...|
|  1.0|[37.0,1.0,0.0,0.0...|
|  1.0|[28.0,5090.0,0.0,...|
+-----+--------------------+
only showing top 10 rows



In [9]:
from pyspark.ml.feature import PCA
bankPCA = PCA(k=3, inputCol="features", outputCol="pcaFeatures")
pcaModel = bankPCA.fit(bankDF)
pcaResult = pcaModel.transform(bankDF).select("label","pcaFeatures")
pcaResult.show(truncate=False)

#Indexing needed as pre-req for Decision Trees
from pyspark.ml.feature import StringIndexer
stringIndexer = StringIndexer(inputCol="label", outputCol="indexed")
si_model = stringIndexer.fit(pcaResult)
td = si_model.transform(pcaResult)
td.collect()


+-----+------------------------------------------------------------+
|label|pcaFeatures                                                 |
+-----+------------------------------------------------------------+
|1.0  |[-2343.0242639663584,58.0195309324747,-0.5351007194406772]  |
|1.0  |[-45.02321764751286,55.973562013159956,-0.52229775673338]   |
|1.0  |[-1270.0168901254456,40.47015758632763,-0.660723951200363]  |
|1.0  |[-2476.022593319605,53.96561320883181,-0.5689309206326586]  |
|1.0  |[-184.0223971014953,53.91583868938551,0.764137862884039]    |
|1.0  |[-0.01742050299985475,41.96349770478308,1.1308880232269072] |
|1.0  |[-830.0231621700161,55.64666140589562,0.7024775001688153]   |
|1.0  |[-545.0248280850174,59.7567438123837,-0.1752810112600166]   |
|1.0  |[-1.015340319497228,36.99772086009115,-0.6722323047147304]  |
|1.0  |[-5090.011162462702,25.858090376679616,-0.2984199243037522] |
|1.0  |[-100.01574051756292,37.924587785230756,-0.1396903517385719]|
|1.0  |[-309.0124102017519,29.8721

[Row(label=1.0, pcaFeatures=DenseVector([-2343.0243, 58.0195, -0.5351]), indexed=1.0),
 Row(label=1.0, pcaFeatures=DenseVector([-45.0232, 55.9736, -0.5223]), indexed=1.0),
 Row(label=1.0, pcaFeatures=DenseVector([-1270.0169, 40.4702, -0.6607]), indexed=1.0),
 Row(label=1.0, pcaFeatures=DenseVector([-2476.0226, 53.9656, -0.5689]), indexed=1.0),
 Row(label=1.0, pcaFeatures=DenseVector([-184.0224, 53.9158, 0.7641]), indexed=1.0),
 Row(label=1.0, pcaFeatures=DenseVector([-0.0174, 41.9635, 1.1309]), indexed=1.0),
 Row(label=1.0, pcaFeatures=DenseVector([-830.0232, 55.6467, 0.7025]), indexed=1.0),
 Row(label=1.0, pcaFeatures=DenseVector([-545.0248, 59.7567, -0.1753]), indexed=1.0),
 Row(label=1.0, pcaFeatures=DenseVector([-1.0153, 36.9977, -0.6722]), indexed=1.0),
 Row(label=1.0, pcaFeatures=DenseVector([-5090.0112, 25.8581, -0.2984]), indexed=1.0),
 Row(label=1.0, pcaFeatures=DenseVector([-100.0157, 37.9246, -0.1397]), indexed=1.0),
 Row(label=1.0, pcaFeatures=DenseVector([-309.0124, 29.872

In [10]:
(trainingData, testData) = td.randomSplit([0.7, 0.3])
trainingData.count()
testData.count()
testData.collect()


[Row(label=0.0, pcaFeatures=DenseVector([-9447.01, 22.0509, 0.9209]), indexed=0.0),
 Row(label=0.0, pcaFeatures=DenseVector([-8121.0242, 56.6222, -0.6864]), indexed=0.0),
 Row(label=0.0, pcaFeatures=DenseVector([-7944.0147, 33.7027, 0.5058]), indexed=0.0),
 Row(label=0.0, pcaFeatures=DenseVector([-7132.0214, 50.0345, 0.6456]), indexed=0.0),
 Row(label=0.0, pcaFeatures=DenseVector([-4692.0253, 60.0437, 0.7558]), indexed=0.0),
 Row(label=0.0, pcaFeatures=DenseVector([-3754.0109, 25.4125, -0.2851]), indexed=0.0),
 Row(label=0.0, pcaFeatures=DenseVector([-3518.0159, 37.5377, 0.5921]), indexed=0.0),
 Row(label=0.0, pcaFeatures=DenseVector([-2990.0159, 37.7567, 0.6005]), indexed=0.0),
 Row(label=0.0, pcaFeatures=DenseVector([-2971.0143, 33.7665, -0.7354]), indexed=0.0),
 Row(label=0.0, pcaFeatures=DenseVector([-2749.0159, 37.8256, -0.1739]), indexed=0.0),
 Row(label=0.0, pcaFeatures=DenseVector([-2657.018, 42.9044, 0.0667]), indexed=0.0),
 Row(label=0.0, pcaFeatures=DenseVector([-2500.0193, 

In [11]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [12]:
rmClassifer = RandomForestClassifier(labelCol="indexed", \
                featuresCol="pcaFeatures")
rmModel = rmClassifer.fit(trainingData)

#Predict on the test data
predictions = rmModel.transform(testData)
predictions.select("prediction","indexed","label","pcaFeatures").collect()
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", \
                    labelCol="indexed",metricName="accuracy")
evaluator.evaluate(predictions)

0.6181384248210023

In [13]:
predictions.groupBy("indexed","prediction").count().show()


+-------+----------+-----+
|indexed|prediction|count|
+-------+----------+-----+
|    1.0|       1.0|  712|
|    0.0|       1.0|  400|
|    1.0|       0.0|  880|
|    0.0|       0.0| 1360|
+-------+----------+-----+

